### this script has been taken from the checkFiltering and DivideTrialsIntoReadingAndWriting script, and so all the filtering is based on the same

In [1]:
%run Functions.py

In [4]:
def GetBlinkPropertiesForEvents_wBlinkBursts(EventTrial, PupilData_df, MissingData_startNlength, BlinkBursts):
    
    timeList = PupilData_df['timeStamp'].tolist()
    
    samplingFrequency = 90
    timeStampStart = BlinkBursts['timeStampStart']
    timeStampEnd = BlinkBursts['timeStampEnd']
    
    
    #blinkTimeStart = [timeStamp[indBlink[0]] for indBlink in MissingData_startNlength]
    #blinkTimeEnd = [timeStamp[indBlink[0]+indBlink[1]] if indBlink[0]+indBlink[1] < len(timeStamp) else timeStamp[-1] \
    #                for indBlink in MissingData_startNlength]
    
    
    
    blinkBurstCountList = list()
    blinkBurstRateList = list()
    
    # for every trial event, find the blinks during that event
    for ind, timeStart in enumerate(EventTrial['start']):
        timeEnd = EventTrial['end'][ind]
        timeTrial = ((timeEnd - timeStart).total_seconds())/60
        #print('')
        #print(ind, timeList.index(timeStart), timeList.index(timeEnd))
            
        trialInd = list()
        for ind2, blinkStart in enumerate(timeStampStart):
            blinkEnd = timeStampEnd[ind2]
            
            if blinkStart > timeStart and blinkEnd < timeEnd:
                #print('add: ', timeList.index(blinkStart), timeList.index(blinkEnd))
                trialInd.append(ind2)
                
                    
        if len(trialInd) > 0:
            if len(trialInd) == 1:
                
                blinkBurstCountList.append(1)
                blinkBurstRateList.append(1/timeTrial)
            else:
                blinkBurstCountList.append(len(trialInd))
                blinkBurstRateList.append(len(trialInd)/timeTrial)
        elif len(trialInd) == 0:
            blinkBurstCountList.append(0)
            blinkBurstRateList.append(0)
            
            
    
    Blink_df = pd.DataFrame(list(zip(blinkBurstCountList, blinkBurstRateList)), columns=['blinkBurstCount'\
            , 'blinkBurstRate'])
    Blink_df = Blink_df.fillna(0)    
    
    return Blink_df

In [ ]:
class DataForEveryTrial:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
    

In [6]:
metricComputed_count = 'BlinkBurstCount_writing'
metricComputed_freq = 'BlinkBurstFrequency_writing'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName_count = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_count +  '.xlsx'
resultFileName_freq = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_freq +  '.xlsx'



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        #if 'af\\1\\2019-01-16-15-43-8_1' not in root:
        #    continue
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
            
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            print(subjAndSessionName)
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            userKeys_new_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root, \
                                                   userKeys_new_wDwellTime)
            
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                blinkCount = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkDurationTotal = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkDurationAverage = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkFrequency = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                interBlinkDuration = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
            
            else:
                # filter the data
                pupilData_filtered, interpolatedIndices, missingData_fromBlinks, missingData_overall, \
                missingDataBlink_startNlength, missingDataAll_startNlength, blinkBursts = FilterPupilSize_wBlinkDataAndBurst(\
                                                gazeLog, timeTyping, subjAndSessionName)
                
                """
                print('missing data ratio: ', sum(missingData_overall)/len(missingData_overall))
                print('interpolated data ratio: ', sum(interpolatedIndices)/len(interpolatedIndices))
                print('missing data from blinks ratio: ', sum(missingData_fromBlinks)/len(missingData_fromBlinks))
                
                blinkDurations = [item[1] for item in missingDataBlink_startNlength]
                fig=plt.figure(), plt.hist(blinkDurations)
                plt.title('blinkDurationHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\blinkDurationHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                missingDataAllDurations = [item[1] for item in missingDataAll_startNlength]
                fig=plt.figure(), plt.hist(missingDataAllDurations)
                plt.title('MissingDataLengthHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\MissingDataLengthHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                
                """
                #print(missingDataBlink_startNlength)
                
                # divide trials into reading and writing
                
                eventReading, eventReading_index = EventPartsFromPupilData(eventTrialsInKeysSelected_reading, pupilData_filtered, root)
                eventWriting, eventWriting_index = EventPartsFromPupilData(eventTrialsInKeysSelected_writing, pupilData_filtered, root)
                
                eventTrial, eventTrial_index = CombineReadingWriting(eventReading, eventWriting, eventReading_index, eventWriting_index)
                
                #print('eventReadingIndex: ', eventReading_index)
                #print('')
                #print('eventTrialIndex:  ', eventTrial_index)
                
                # compute blink durations for all trials
                blink = GetBlinkPropertiesForEvents_wBlinkBursts(eventWriting, pupilData_filtered, missingDataBlink_startNlength,\
                                                   blinkBursts)
                
                blinkCount = blink['blinkBurstCount'].tolist()
                blinkFrequency = blink['blinkBurstRate'].tolist()
                
                
            if '1stPart' in root:
                print('1stPart')
                blinkCount1 = blinkCount
                blinkFrequency1 = blinkFrequency
                
                continue
            
            if '2ndPart' in root:
                print('2ndPart')
                blinkCount2 = blinkCount
                blinkFrequency2 = blinkFrequency
                
                blinkCount = blinkCount1 + blinkCount2
                blinkFrequency = blinkFrequency1 + blinkFrequency2
                
                blinkCount1 = list()
                blinkFrequency1 = list()
                
            
            # save the blink count
            dataToSave_count = DataForEveryTrial()
            dataToSave_count.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_count.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_count.sessionNumber = subjAndSessionName[-1]
            dataToSave_count.variable = metricComputed_count
            dataToSave_count.dataForTrial = blinkCount
            dataToSave_count.resultPathName = resultFileName_count
            
            print(dataToSave_count.printInfo())
            
            
            
            # save the blink frequency
            dataToSave_freq = DataForEveryTrial()
            dataToSave_freq.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_freq.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_freq.sessionNumber = subjAndSessionName[-1]
            dataToSave_freq.variable = metricComputed_freq
            dataToSave_freq.dataForTrial = blinkFrequency
            dataToSave_freq.resultPathName = resultFileName_freq
            
            print(dataToSave_freq.printInfo())
            
            
            #dataToSave_count.AddToFile()
            #dataToSave_freq.AddToFile()
            
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
ac__1__2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
  subjectID block session  trial  BlinkBurstCount_writing
0        ac     1       1      0                        2
1        ac     1       1      1                        1
2        ac     1       1      2                        0
3        ac     1       1      3                        0
4        ac     1       1      4                        0
  subjectID block session  trial  BlinkBurstFrequency_writing
0        ac     1       1      0                     1.279392
1        ac     1       1      1                     0.935813
2        ac     1       1      2                     0.000000
3        ac     1       1      3                     0.000000
4        ac     1       1      4                     0.000000
subject path E:\Data\Data\ac\1\2019-02-11-11-33-32_2
ac__1__2019-02-11-11-33-32_2
subject and session name:  ac__1__2019-02-11-11-33-32_2
  su

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-0bfd94f27860>", line 154, in <module>
    gazeLog, timeTyping, subjAndSessionName)
  File "<ipython-input-3-c12689c2fd9a>", line 7, in FilterPupilSize_wBlinkDataAndBurst
    timeGazeLog = timeConversion(timeStrGazeLog)
  File "C:\DTU\PythonScripts\Analysis_Expt_201901\Functions.py", line 556, in timeConversion
    timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
  File "C:\ProgramData\Anaconda3\lib\_strptime.py", line 577, in _strptime_datetime
    tt, fraction, gmtoff_fraction = _strptime(data_string, format)
  File "C:\ProgramData\Anaconda3\lib\_strptime.py", line 324, in _strptime
    if not isinstance(arg, str):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (mos

KeyboardInterrupt: 